In [ ]:
import yaml
import alfworld
import alfworld.agents.environment
with open('base_config.yaml') as reader:
    config = yaml.safe_load(reader)
    
print('hello wolrd')
split = "eval_out_of_distribution"

env = getattr(alfworld.agents.environment, config["env"]["type"])(config, train_eval=split)
env = env.init_env(batch_size=1)

In [ ]:
from discussion_agents.cog.prompts.react import REACT_ALFWORLD_PROMPTS_EXAMPLE

In [ ]:

d = REACT_ALFWORLD_PROMPTS_EXAMPLE

In [ ]:
ob, info = env.reset()
ob = '\n'.join(ob[0].split('\n\n')[1:])
name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1])

In [ ]:
ob , name

In [ ]:
prefixes = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}

In [ ]:
for i, (k, v) in enumerate(prefixes.items()):
    if name.startswith(k):
        prompt = 'Interact with a household to solve a task. Here are two examples.\n' + d[f'react_{v}_1'] + '\nHere is the task.\n'
        print(k, v)
        

In [ ]:
from langchain.chat_models.openai import ChatOpenAI
from discussion_agents.cog.agent.react import ReActAgent


api_key = 

llm = ChatOpenAI(openai_api_key = api_key)

test_run = ReActAgent(llm = llm )

In [ ]:
question = 'Brad Wilk died before being a drummer for Greta.'

In [ ]:
from discussion_agents.cog.prompts.react import REACT_WEBTHINK_SIMPLE3_FEVER_EXAMPLES

In [ ]:
print(test_run.generate(question=ob , examples=prompt,env=env))

In [ ]:
print(test_run.memory.load_memories()['scratchpad'])

In [ ]:
from discussion_agents.cog.prompts.react import REACT_ALFWORLD_INSTRUCTION
from discussion_agents.cog.functional.react import _prompt_agent

In [ ]:
action = _prompt_agent(
    llm=test_run.llm,
    question=ob,
    scratchpad=test_run.memory.load_memories()["scratchpad"],
    examples=prompt,
    instruction=REACT_ALFWORLD_INSTRUCTION
)

In [ ]:
check_type(REACT_WEBTHINK_SIMPLE3_FEVER_EXAMPLES)

In [ ]:
def check_type(examples: str = None) -> str:
    lines = examples.split('\n')
    lines = [line for line in lines if line.strip()] 
    checkword = lines[0].split()[0]
    if checkword == 'Question:':
        return 'HOTPOTQA'
    else:
        checkword = lines[0].split()[0]
        if checkword == 'Claim:':
            return 'FEVER'
        line = lines[2]
        print(line)
        if 'Your task is to:' in line :
            return 'ALFWORLD'
    return ValueError('Wrong Examples')

In [ ]:
print(env)

In [ ]:
benchmark_type = check_type(prompt)    

In [ ]:
print(prompt)

In [ ]:
observation, reward, done, info = env.step(['put tomato in microwave'])

In [ ]:
done

In [ ]:
observation, reward, done, info = env.step(['put tomato in/on microwave'])

In [ ]:
from typing import Any, Dict, List, Optional

In [ ]:
def testing_generate(env : Any = None, action : str = None):
    observation, reward, done, info = env.step([action])
    observation, reward, done = observation[0], info['won'][0], done[0]
    return observation , reward , done

In [ ]:
env